## Memory

In [3]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
from langchain.memory import ChatMessageHistory, ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.chains import ConversationChain

load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

In [5]:
# message history
history = ChatMessageHistory()

history.add_user_message("Buenos dias!")
history.add_ai_message("hello!")
history.add_user_message("Whats your name?")
history.add_ai_message("My name is GIGACHAT")
history.messages

[HumanMessage(content='Buenos dias!'),
 AIMessage(content='hello!'),
 HumanMessage(content='Whats your name?'),
 AIMessage(content='My name is GIGACHAT')]

In [8]:
# memory
# History keeps all messages between the user and AI intact. History is what the user sees in the UI.
# It represents what was actually said. Memory keeps some information, which is presented to the LLM to make it behave as if it "remembers" the conversation.
memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("Buenos dias!")
memory.chat_memory.add_ai_message("Hello!")
memory.chat_memory.add_user_message("Whats your name?")
memory.chat_memory.add_ai_message("My name is GIGACHAT")
memory.load_memory_variables({})

{'history': 'Human: Buenos dias!\nAI: hello!\nHuman: Whats your name?\nAI: My name is GIGACHAT'}

In [12]:
llm = OpenAI(temperature=0)
conversation = ConversationChain(
    llm=llm, verbose=True, memory=ConversationBufferMemory()
)
conversation.predict(input="Buenos Dias!") # will it response in different language?



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Buenos Dias!
AI:

> Finished chain.


" Hello! It's currently 9:00 AM here in my location. I am an AI programmed to respond to human interactions. How can I assist you today?"

## Summary

In [11]:
!pip install tiktoken

In [21]:
from langchain.memory import ConversationSummaryBufferMemory

previous_output_review = "Rdr2 is an experience, as it's more than just a video game. It's like being in one long Oscar winning movie. Script, acting, storyline are immense, it's quite unbelievable really. You are living as a cowboy day in day out eating, bathing, sleeping, everything in a huge openworld."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=1-0)
memory.save_context(
    {"input": f"Could you analyze review for me {previous_output_review}?"},
    {"output": "Sure, no problem"},
)

In [22]:
conversation = ConversationChain(llm=llm, memory=memory, verbose=True)

In [23]:
conversation.predict(input="Thank you")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human asks the AI to analyze a review for the game Rdr2, which is described as an immersive experience similar to living in an Oscar-winning movie. The AI is asked to analyze the script, acting, and storyline, which are all praised as immense and unbelievable. The game allows players to live as a cowboy in a huge open world, performing daily tasks such as eating, bathing, and sleeping.
AI: Sure, no problem
Human: Thank you
AI:

> Finished chain.


" You're welcome! So, let me start by saying that Rdr2 stands for Red Dead Redemption 2, which is a popular action-adventure game developed by Rockstar Games. It was released in 2018 and has received critical acclaim for its immersive gameplay and detailed open world. The game is set in the late 19th century in the American Old West and follows the story of Arthur Morgan, a member of the Van der Linde gang. The script for the game was written by Dan Houser, who is known for his work on other successful Rockstar titles such as Grand Theft Auto V. The acting in Rdr2 is top-notch, with motion capture technology used to bring the characters to life. The main storyline is praised for its depth and emotional impact, with players having to make tough decisions that affect the outcome of the game. As for the daily tasks, they add to the immersion of the game and make players feel like they are truly living in the Old West. Is there anything else you would like me to analyze about the game?"